In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import sys
import time

sys.path.append('..')
from genotype_evaluator import GenotypeEvaluator

%matplotlib inline

## Pbil implementation

In [7]:
def random_individual(probs):
    x = np.empty(probs.shape[0], dtype=np.int32)
    for i in range(probs.shape[0]):
        x[i] = np.random.choice(3, p=[probs[i][0], probs[i][1], probs[i][2]])
    return x


def random_population(probs, population_size):
    population = np.empty((population_size, probs.shape[0]), dtype=np.int32)
    for i in range(population_size):
        population[i] = random_individual(probs)
    return population


def get_updated_probabilities(probs, learning_rate, increased_index):
    updated_probs = probs.copy()
    updated_probs[increased_index] = (1 - learning_rate) * probs[increased_index] + learning_rate
    decreased_indexes = list(set(range(len(probs))).difference([increased_index]))
    decreased_elements_sum = np.sum(updated_probs[decreased_indexes])
    if decreased_elements_sum == 0:
        return updated_probs
    excess_sum = np.sum(updated_probs) - 1
    updated_probs[decreased_indexes] *= (decreased_elements_sum - excess_sum) / decreased_elements_sum
    return updated_probs 


def pbil(objective_function, dim_size, population_size, learning_rate, mutation_prob, mutation_noise, max_time):
    p = np.ones((dim_size, 3)) / 3
    start_time = time.time()
    populations = []
    while time.time() - start_time < max_time:
        samples = random_population(p, population_size)
        populations.append(samples)
        score_values = objective_function(samples)
        print("Average score values: {}, best score value: {}".format(np.mean(score_values), np.max(score_values)))
        best_individual = samples[np.argmax(score_values)]
        for k in range(dim_size):
            p[k] = get_updated_probabilities(p[k], learning_rate, best_individual[k])
        for k in range(dim_size):
            if np.random.rand() < mutation_prob:
                increased_index = np.random.choice(3)
                p[k] = get_updated_probabilities(p[k], mutation_noise, increased_index)
    return populations, p

## Rounded board evaluator

In [2]:
genotype_evaluator = GenotypeEvaluator('../clusters/human100_cluster.c', ['../boards/round_board.b'])

In [8]:
rb_populations, rb_probs = pbil(genotype_evaluator.evaluate_genotypes, 100, 100, 0.1, 0.05, 0.01, 3600)

Average score values: 350.23, best score value: 674.0
Average score values: 330.11, best score value: 2111.0
Average score values: 342.0, best score value: 734.0
Average score values: 388.85, best score value: 2112.0
Average score values: 344.06, best score value: 640.0
Average score values: 389.21, best score value: 690.0
Average score values: 439.68, best score value: 2111.0
Average score values: 370.83, best score value: 2111.0
Average score values: 414.13, best score value: 2110.0
Average score values: 439.49, best score value: 2111.0
Average score values: 434.49, best score value: 2008.0
Average score values: 459.95, best score value: 2111.0
Average score values: 481.03, best score value: 2111.0
Average score values: 477.21, best score value: 2111.0
Average score values: 482.77, best score value: 2111.0
Average score values: 549.25, best score value: 2111.0
Average score values: 593.5, best score value: 2111.0
Average score values: 654.96, best score value: 2111.0
Average score va

In [31]:
with open('../individuals/rb_pbil_gen5.i', 'wb') as file_stream:
    best_index = np.argmax(genotype_evaluator.evaluate_genotypes(rb_populations[5]))
    gen5_individual = genotype_evaluator.individual_factory.create_individual(rb_populations[5][best_index])
    gen5_individual.marker_manager.game_state_marker_lines = {}
    pickle.dump(gen5_individual, file_stream)
with open('../individuals/rb_pbil_best.i', 'wb') as file_stream:
    best_index = np.argmax(genotype_evaluator.evaluate_genotypes(rb_populations[-1]))
    best_individual = genotype_evaluator.individual_factory.create_individual(rb_populations[-1][best_index])
    best_individual.marker_manager.game_state_marker_lines = {}
    pickle.dump(best_individual, file_stream)